In [14]:
#from google.colab import drive
#drive.mount('/gdrive')

In [12]:
# imports

import pandas as pd
#from google.colab import files
from os import listdir
from os.path import isfile, join
import re
from collections import Counter
from PIL import Image
import numpy as np
from keras.models import Model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from matplotlib.image import imread
from random import shuffle
from math import ceil

In [13]:
# variaveis

img_to_load = 300 # imagens a serem carregadas por vez (carregando as 10000 de uma vez sobrecarrega o sistema)
crop_size = 150 # altura e largura da imagem a ser enviada para o modelo

In [14]:
#função para pegar só o centro da imagem

def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [15]:
# usar se estiver usando google colab, isso descompacta um arquivo no diretório do notebook

#!unzip regression_model_0_0070226178504526615.zip

In [16]:
# carrega um modelo salvo anteriormente

c_model = load_model('classification_model')
r_model = load_model('regression_model')

In [17]:
public_path = 'public_test/'
public_files = [f'{public_path}{f}' for f in listdir(public_path) if isfile(join(public_path, f))]
print(len(public_files))
public_files[:5]

1502


['public_test/001ec8f4d2679f2899a63e4b2597c0a10bd9393f.png',
 'public_test/00327390bfd6101b87106e96e0efc1c938690e1f.png',
 'public_test/00573dde8addbac780cfa35a55a790fa1aa45178.png',
 'public_test/00cb5b91a4acdfaa2f55dc2266e598ff06f9209e.png',
 'public_test/00de1146bdcbfb02ed5f0e13fd6b9fe62157c0b2.png']

In [18]:
private_path = 'private_test/'
private_files = [f'{private_path}{f}' for f in listdir(private_path) if isfile(join(private_path, f))]
print(len(private_files))
private_files[:5]

15062


['private_test/0002894871bb30af2670648c58b2506e9801a321.png',
 'private_test/000a95ca23df016a149ff9af94b6e9d8633d6691.png',
 'private_test/000b4407dee7dad2ba46586e6ba1264f45965e51.png',
 'private_test/000c45d729066044e3dd3f885fddf013b845b6d7.png',
 'private_test/0011f80e44faa713714ead6aeab3b2f9db54aebd.png']

In [19]:
all_test_files = public_files + private_files
len(all_test_files)

16564

In [21]:
#img_to_load = 10

df = pd.DataFrame()

for i in range(ceil(len(all_test_files)/img_to_load)):
    begin = i*img_to_load
    limit = min(i*img_to_load + img_to_load, len(all_test_files))
    if begin >= limit:
        break
    
    X_str = all_test_files[begin:limit]
    X = [crop_center(imread(file), crop_size, crop_size) for file in all_test_files[begin:limit]]
    X = np.stack(X, axis=0)
    X = X.reshape((X.shape + (1,)))
    print(f'loaded {begin} to {limit}')
    
    c_predictions = (c_model.predict(X) > 0.5).astype("int32")
    r_predictions = (r_model.predict(X)) * 30

    list_dict = [{'id': re.findall(r"(?<=_test\/).*(?=\.png)", X_str[i])[0], 
                  'classification_predictions': c_predictions[i][0], 
                  'regression_predictions': r_predictions[i][0]} for i in range(len(X))]
    df_aux = pd.DataFrame(list_dict)
    df = pd.concat([df, df_aux])
    print(len(df))

loaded 0 to 300
300
loaded 300 to 600
600
loaded 600 to 900
900
loaded 900 to 1200
1200
loaded 1200 to 1500
1500
loaded 1500 to 1800
1800
loaded 1800 to 2100
2100
loaded 2100 to 2400
2400
loaded 2400 to 2700
2700
loaded 2700 to 3000
3000
loaded 3000 to 3300
3300
loaded 3300 to 3600
3600
loaded 3600 to 3900
3900
loaded 3900 to 4200
4200
loaded 4200 to 4500
4500
loaded 4500 to 4800
4800
loaded 4800 to 5100
5100
loaded 5100 to 5400
5400
loaded 5400 to 5700
5700
loaded 5700 to 6000
6000
loaded 6000 to 6300
6300
loaded 6300 to 6600
6600
loaded 6600 to 6900
6900
loaded 6900 to 7200
7200
loaded 7200 to 7500
7500
loaded 7500 to 7800
7800
loaded 7800 to 8100
8100
loaded 8100 to 8400
8400
loaded 8400 to 8700
8700
loaded 8700 to 9000
9000
loaded 9000 to 9300
9300
loaded 9300 to 9600
9600
loaded 9600 to 9900
9900
loaded 9900 to 10200
10200
loaded 10200 to 10500
10500
loaded 10500 to 10800
10800
loaded 10800 to 11100
11100
loaded 11100 to 11400
11400
loaded 11400 to 11700
11700
loaded 11700 to 1200

In [31]:
df = df.sort_values(by=['id'])
df.to_csv('predictions.csv', index=False)

In [23]:
len(df.index)

16564

In [26]:
test['id'].value_counts()

9105cd8246e5d52859f0b8e149e4716bc651800c    1
47e22be57861bb9235cc98b6efe9d4d9ec5f0e3f    1
c1af5e23e5ee51bb6ed07648a852b227b95d6242    1
977f0dc8754d5ed3f0c249d973919a6be0d7c0b1    1
1598beef1def969362868c27d2047dd55645821a    1
                                           ..
c2c93b456a28ff2d443a843f1cf0176c1fde254b    1
58ee76462562c96d6dd9e16d6d13f2dbdcb33ac0    1
4ceb20813e2864c76faacfe4b64603fc5b25f096    1
92a947a7fb6abe81db45bca2f5c79a4dfa4779c2    1
867d741d64c063f6b5596bbf5a6876cccdd2ce77    1
Name: id, Length: 16564, dtype: int64

In [30]:
df

,id,classification_predictions,regression_predictions
2,0002894871bb30af2670648c58b2506e9801a321,0,28.810635
3,000a95ca23df016a149ff9af94b6e9d8633d6691,1,24.765879
4,000b4407dee7dad2ba46586e6ba1264f45965e51,1,23.256393
5,000c45d729066044e3dd3f885fddf013b845b6d7,0,29.423479
6,0011f80e44faa713714ead6aeab3b2f9db54aebd,1,7.986916
...,...,...,...
61,ffdfd25b2f522ba2312dda90189ec48205578199,0,27.798077
62,ffe29ac691148f84c7a7759e687aa6c2f0eac11a,1,25.308645
0,ffedc12ec5c142dae2086408b778cf0db2ff944e,0,2.953554
63,fff423086a80114984b647af69f710cf3ef844f9,1,3.372553
